# Imports

In [2]:
import pandas as pd

# GLM Table

In [6]:
# read csv
all_significant_contrasts = pd.read_csv('processed_data/glm/all_significant_contrasts.csv')

# remove the P>|z| and Significant column
all_significant_contrasts = all_significant_contrasts.drop(columns=['P>|z|', 'Significant', 'Chroma', '[0.025', '0.975]', 'mode'])

# rename the columns and capitalize the first letter of each word, except for 'z' and 'p_value_fdr'
all_significant_contrasts.columns = [col.replace('_', ' ').title() if col not in ['z', 'p_value_fdr'] else col for col in all_significant_contrasts.columns]

# rename the p_value_fdr column to $p_{fdr}$
all_significant_contrasts.rename(columns={'p_value_fdr': '$p_{fdr}$'}, inplace=True)

# rename Contrast Pair to 'Contrast'
all_significant_contrasts.rename(columns={'Contrast Pair': 'Contrast'}, inplace=True)

# rename 'z' to '$z$'
all_significant_contrasts.rename(columns={'z': '$z$'}, inplace=True)

# reorder the columns
all_significant_contrasts = all_significant_contrasts[['Contrast', 'Region', 'Ch Name', 'Coef.', 'Std.Err.', '$z$', '$p_{fdr}$']]

caption = "Table of contrast results from the GLM analysis. "

# save as a latex table
with open('processed_data/glm/all_significant_contrasts.tex', 'w') as f:
    f.write(all_significant_contrasts.to_latex(index=False, float_format="%.3f", escape=False, column_format='|c|c|c|c|c|c|c|', longtable=True, caption=caption))

# remove every underscore from the latex table
with open('processed_data/glm/all_significant_contrasts.tex', 'r') as f:
    lines = f.readlines()
with open('processed_data/glm/all_significant_contrasts.tex', 'w') as f:
    for line in lines:
        if "fdr" in line:
            f.write(line.replace('>', '$>$'))
        else:
            f.write(line.replace('_', ' ').replace('>', '$>$'))

# Functional Connectivity Table

In [4]:
roi_t_vals = pd.read_csv(f'processed_data\\spectral_connectivity_time\\group_level_t_tests_roi_t_vals.csv')
roi_t_vals = roi_t_vals[roi_t_vals['Mode'] != 'all']

roi_p_vals = pd.read_csv(f'processed_data\\spectral_connectivity_time\\group_level_t_tests_roi_p_vals.csv')
roi_p_vals = roi_p_vals[roi_p_vals['Mode'] != 'all']

# remove the Mode column
roi_t_vals = roi_t_vals.drop(columns=['Mode'])
roi_p_vals = roi_p_vals.drop(columns=['Mode'])

# rename the Condition column to 'Contrast'
roi_t_vals.rename(columns={'Condition': 'Contrast'}, inplace=True)
roi_p_vals.rename(columns={'Condition': 'Contrast'}, inplace=True)

# average the t-values across all ROIs for each contrast
roi_t_vals['Mean $t$ value'] = roi_t_vals.iloc[:, 1:].mean(axis=1)

# sum the p-values across all ROIs for each contrast
roi_p_vals['Sum of Sig. diff channels'] = roi_p_vals.iloc[:, 1:].sum(axis=1)

# reorder the columns
roi_t_vals = roi_t_vals[['Contrast', 'Mean $t$ value']]
roi_p_vals = roi_p_vals[['Contrast', 'Sum of Sig. diff channels']]

# merge the two dataframes
roi_vals = pd.merge(roi_t_vals, roi_p_vals, on='Contrast')

# Each contrast string is in one of the following formats:
# 'Condition1_Condition2' or 'Condition1_Condition2_Condition3_Condition4'
# In the first case, we want to replace the underscore with ' > ' and in the second case, we want to replace the first and last underscores with ' ' and the middle underscores with ' > '.
def format_contrast_string(contrast):
    parts = contrast.split('_')
    if len(parts) == 2:
        return f"{parts[0]} > {parts[1]}"
    elif len(parts) == 4:
        return f"{parts[0]} {parts[1]} > {parts[2]} {parts[3]}"
    else:
        return contrast
    
# apply the formatting function to the Contrast column
roi_vals['Contrast'] = roi_vals['Contrast'].apply(format_contrast_string)

caption = "Group level $t$-tests and Sum of Significantly different channels averaged across ROIs. "

# save as a latex table
with open('processed_data/spectral_connectivity_time/group_level_t_tests_roi_vals.tex', 'w') as f:
    f.write(roi_vals.to_latex(index=False, float_format="%.3f", escape=False, column_format='|l|c|c|', longtable=True, caption=caption))

# remove every underscore from the latex table
with open('processed_data/spectral_connectivity_time/group_level_t_tests_roi_vals.tex', 'r') as f:
    lines = f.readlines()
with open('processed_data/spectral_connectivity_time/group_level_t_tests_roi_vals.tex', 'w') as f:
    for line in lines:
        f.write(line.replace('_', '\_').replace('>', '$>$'))

# ANOVA Table

In [5]:
# read csv
anova_table = pd.read_csv('processed_data/behavioural_responses/anova_table.csv')

caption = 'Two-way ANOVA results for the effect of Face Type and Emotion and their interaction on the correct responses. '

# set nan's to empty strings for better LaTeX formatting
anova_table = anova_table.fillna('')

# rename 'F' to '$F$' and 'PR(>F)' to 'PR(>$F$)'
anova_table.rename(columns={'F': '$F$', 'PR(>F)': 'PR(>$F$)'}, inplace=True)

# rename the first column to ''
anova_table.rename(columns={anova_table.columns[0]: ''}, inplace=True)

# save the anova table to a latex file
with open('processed_data/behavioural_responses/anova_table.tex', 'w') as f:
    f.write(anova_table.to_latex(index=False, float_format="%.5f", escape=False, column_format='|l|c|c|c|c|', longtable=True, caption=caption))

# remove every underscore from the latex table
with open('processed_data/behavioural_responses/anova_table.tex', 'r') as f:
    anova_table_latex = f.read().replace('_', '\\_').replace('>', '\\textgreater{}')
with open('processed_data/behavioural_responses/anova_table.tex', 'w') as f:
    f.write(anova_table_latex)